# Run Diabetes Training Script (SDK v2) — with End‑to‑End AML Features

This notebook demonstrates a complete **Azure Machine Learning (SDK v2)** workflow for the classic **diabetes regression** example:

- Connect to your **Azure ML workspace** using `MLClient` and `DefaultAzureCredential`.  
- Create or attach an **AmlCompute** cluster.  
- Load the **scikit‑learn diabetes dataset**, train a **Ridge regression** model, and log metrics with **MLflow**.  
- **Register** the trained **MLflow** model to the AML **model registry**.  
- **Deploy** the model to a **managed online endpoint** and **invoke** it for real‑time scoring.  
- (Optional) **Clean up** endpoint resources.

> This notebook fills typical “missing functionalities” seen in minimal examples: robust workspace connection, compute provisioning, MLflow tracking, model registration, endpoint creation, invocation testing, and cleanup.

**Prerequisites**
- Azure ML workspace & permissions.
- Python packages: `azure-ai-ml`, `azure-identity`, `mlflow`, `azureml-mlflow`, `scikit-learn`.



In [ ]:

# If you run on an Azure ML compute instance, many of these are preinstalled.
# You may safely re-run to ensure versions are available.

!pip -q install --upgrade azure-ai-ml azure-identity mlflow azureml-mlflow scikit-learn


In [ ]:

import os
import json
import datetime

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute, Model, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import ModelType

import mlflow
from mlflow.tracking import MlflowClient
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Workspace connection: prefer DefaultAzureCredential, fallback to InteractiveBrowserCredential
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    credential = InteractiveBrowserCredential()

# Try to connect using config.json; if not present, set manual values below.
try:
    ml_client = MLClient.from_config(credential)
    print(f"Connected to workspace: {ml_client.workspace_name}")
except Exception as ex:
    print("No config.json found or default auth failed. Please set workspace details manually.")
    SUBSCRIPTION_ID = "<YOUR_SUBSCRIPTION_ID>"
    RESOURCE_GROUP  = "<YOUR_RESOURCE_GROUP>"
    WORKSPACE_NAME  = "<YOUR_WORKSPACE_NAME>"
    ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, WORKSPACE_NAME)
    print(f"Connected to workspace: {WORKSPACE_NAME}")

# For MLflow tracking to AML workspace
tracking_uri = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(tracking_uri)
print("MLflow tracking URI:", tracking_uri)


In [ ]:

from azure.core.exceptions import ResourceNotFoundError

COMPUTE_NAME = "cpu-diabetes"
VM_SIZE = "STANDARD_DS3_v2"

try:
    _ = ml_client.compute.get(COMPUTE_NAME)
    print(f"Found existing compute: {COMPUTE_NAME}")
except ResourceNotFoundError:
    print(f"Creating compute cluster '{COMPUTE_NAME}'...")
    compute = AmlCompute(
        name=COMPUTE_NAME,
        size=VM_SIZE,
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=120,
    )
    ml_client.begin_create_or_update(compute).result()
    print("Compute created.")


In [ ]:

# Load scikit-learn diabetes dataset
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


In [ ]:

# Train a Ridge regression model and log metrics & model to MLflow
alpha = 0.1
run_name = f"diabetes-ridge-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"

with mlflow.start_run(run_name=run_name) as run:
    model = Ridge(alpha=alpha).fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    rmse = mean_squared_error(y_test, preds, squared=False)

    # Log params & metrics
    mlflow.log_param("alpha", alpha)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("rmse", rmse)

    # Log model in MLflow format
    mlflow.sklearn.log_model(model, artifact_path="model")

    run_id = run.info.run_id
    print("Run ID:", run_id)
    print("R2:", r2, "RMSE:", rmse)


In [ ]:

# The MLflow model is stored under the run's artifacts path 'model'.
# We can register it to AML registry using the MLflow URI or by copying locally.

mlflow_client = MlflowClient()

# Download artifacts locally (optional but convenient)
from mlflow.artifacts import download_artifacts
local_dir = download_artifacts(run_id=run_id, artifact_path="model")
print("Downloaded MLflow model to:", local_dir)

# Register as an AML model (type=MLFLOW) from local path
registered = ml_client.models.create_or_update(
    Model(
        name="diabetes-ridge-mlflow",
        path=local_dir,
        type=ModelType.MLFLOW,
        description="Ridge regression MLflow model for diabetes progression",
        tags={"sample": "diabetes", "framework": "sklearn"}
    )
)
print("Registered model:", registered.id)


In [ ]:

# Create a managed online endpoint and deploy the registered MLflow model.

endpoint_name = "diabetes-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    auth_mode="key",
    description="Diabetes ridge regression (MLflow)"
)
ml_client.begin_create_or_update(endpoint).result()
print("Endpoint created:", endpoint_name)

# Create a deployment named 'blue'
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=registered.id,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(deployment).result()

# Route 100% traffic to 'blue'
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

# Show scoring URI and keys
details = ml_client.online_endpoints.get(endpoint_name)
print("Scoring URI:", details.scoring_uri)
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
print("Primary Key:", keys.primary_key)


In [ ]:

# Prepare a small test payload: the diabetes dataset has 10 numerical features.
# We'll send a few rows from X_test.

import pandas as pd

sample_rows = 5
payload = {"inputs": pd.DataFrame(X_test[:sample_rows]).to_dict(orient="records")}

# Save request to JSON
with open("sample_request.json", "w") as f:
    json.dump(payload, f)

# Invoke the endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="sample_request.json"
)
print("Response:", response)


In [ ]:

# Optional cleanup: delete endpoint when finished
# Uncomment to delete
# ml_client.online_endpoints.begin_delete(name=endpoint_name)
# print("Deleted endpoint:", endpoint_name)
